In [ ]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import pickle
from sklearn.model_selection import GridSearchCV

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../Libraries/")
import utils

# Functions for the dynamic and static models

#### Dynamic model

In [ ]:
def build_dynamic_model(n_timesteps, mask_value, n_dynamic_features, 
                hidden_layer_size, dropout_rate, lr_scheduler):
    # Dynamic preprocessing.
    dynamic_input = tf.keras.layers.Input(shape=(n_timesteps, n_dynamic_features,))
    masked = tf.keras.layers.Masking(mask_value=mask_value)(dynamic_input)
    gru_encoder = tf.keras.layers.GRU(
        hidden_layer_size,
        dropout=dropout_rate,
        return_sequences=False,
        activation='tanh',
        use_bias=False
    )(masked)
    
    # Concatenation
    output = tf.keras.layers.Dense(1, activation="sigmoid")(gru_encoder)
    
    model = tf.keras.Model([dynamic_input], [output])
    customized_loss = utils.weighted_binary_crossentropy(hyperparameters)
    myOptimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
    model.compile(loss=customized_loss, optimizer=myOptimizer)
    
    return model

#### Static model

In [ ]:
def build_static_model(n_timesteps, mask_value, n_static_features, 
                hidden_layer_size, dropout_rate, lr_scheduler):
    # Static preprocessing.
    static_input = tf.keras.layers.Input(shape=(n_static_features))
    hidden_layer = tf.keras.layers.Dense(
        hidden_layer_size,
        activation='tanh'
    )(static_input)
    
    # Concatenation
    output = tf.keras.layers.Dense(1, activation="sigmoid")(hidden_layer)
    
    model = tf.keras.Model([static_input], [output])
    customized_loss = utils.weighted_binary_crossentropy(hyperparameters)
    myOptimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
    model.compile(loss=customized_loss, optimizer=myOptimizer)
    
    return model

In [ ]:
def run_network(X_train, X_train_static, y_train, X_val, X_val_static, y_val, hyperparameters, seed, model_type='dynamic'):
    model = None
    if model_type == 'dynamic':
        model = build_dynamic_model(
            hyperparameters["n_timesteps"], hyperparameters["maskValue"], hyperparameters["n_dynamic_features"], 
            hyperparameters["layers"], hyperparameters["dropout_rate"], hyperparameters["lr_scheduler"]
        )
    elif model_type == 'static':
        model = build_static_model(
            hyperparameters["n_timesteps"], hyperparameters["maskValue"], hyperparameters["n_static_features"], 
            hyperparameters["layers"], hyperparameters["dropout_rate"], hyperparameters["lr_scheduler"]
        )
    else:
        raise ValueError(f"Unknown model_type: {model_type}. Please use 'static' or 'dynamic'.")
    
    try:
        earlystopping = tf.keras.callbacks.EarlyStopping(
            monitor=hyperparameters["monitor"],
            min_delta=hyperparameters["mindelta"],
            patience=hyperparameters["patience"], 
            restore_best_weights=True,
            mode="min"
        )

        # Fit the model based on the selected model_type
        if model_type == 'dynamic':
            hist = model.fit(
                x=X_train, y=y_train,
                validation_data=(X_val, y_val),
                callbacks=[earlystopping], batch_size=hyperparameters['batch_size'], epochs=hyperparameters['epochs'],
                verbose=hyperparameters['verbose'])
            
        elif model_type == 'static':
            hist = model.fit( x=[X_train_static.values], y=y_train,
                             validation_data=([X_val_static.values], y_val),
                             callbacks=[earlystopping], batch_size=hyperparameters['batch_size'], epochs=hyperparameters['epochs'], verbose=0
                            )

        return model, hist, earlystopping
    except KeyboardInterrupt:
        print('Training duration (s):', time.time() - global_start_time)
        return model, None, None


In [ ]:
def myCVGrid(hyperparameters, dropout, lr_scheduler, layers, split, seed, model_type):
    bestHyperparameters = {}
    bestMetricDev = np.inf

    for k in range(len(dropout)):
        for l in range(len(layers)):
            for m in range(len(lr_scheduler)):
                v_early = []
                v_metric_dev = []
                v_hist = []
                v_val_loss = []

                hyperparameters_copy = hyperparameters.copy()
                hyperparameters_copy['dropout_rate'] = dropout[k]
                hyperparameters_copy['layers'] = layers[l]
                hyperparameters_copy['lr_scheduler'] = lr_scheduler[m]
                
                for n in range(5):

                    X_train = np.load("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                          "/X_train_tensor_" + str(n)+ ".npy")
                    X_train_static = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                             "/X_train_static_" + str(n)+ ".csv", index_col=0)
                    y_train = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                          "/y_train_" + str(n)+ ".csv",
                                         index_col=0)
                    X_val = np.load("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                        "/X_val_tensor_" + str(n)+ ".npy")
                    X_val_static = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                             "/X_val_static_" + str(n)+ ".csv", index_col=0)
                    y_val = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) +
                                        "/y_val_" + str(n)+ ".csv",
                                       index_col=0)

                    utils.reset_keras()
                    model, hist, early = run_network(
                        X_train, X_train_static, 
                        y_train,
                        X_val, X_val_static, 
                        y_val,
                        hyperparameters_copy,  
                        seed,
                        model_type
                    )
                                        
                    v_early.append(early)
                    v_hist.append(hist)
                    v_val_loss.append(np.min(hist.history["val_loss"]))
                    
                metric_dev = np.mean(v_val_loss)
                if metric_dev < bestMetricDev:
                    bestMetricDev = metric_dev
                    bestHyperparameters = {
                        'dropout_rate': dropout[k],
                        'layers': layers[l],
                        'lr_scheduler': lr_scheduler[m]
                    }

    return bestHyperparameters, X_train, y_train, X_train_static, X_val, y_val, X_val_static

# Functions for the LR model

In [ ]:
def build_LR(hyperparameters):
    # Static preprocessing.
    static_input = tf.keras.layers.Input(shape=(2))
    # Output
    output = tf.keras.layers.Dense(1, activation="sigmoid")(static_input)
    
    model = tf.keras.Model([static_input], [output])
    customized_loss = utils.weighted_binary_crossentropy(hyperparameters)
    myOptimizer = tf.keras.optimizers.Adam(learning_rate=hyperparameters["lr_scheduler"])
    model.compile(loss=customized_loss, optimizer=myOptimizer)
    
    return model

In [ ]:
def run_network_LR(X_train, y_train, X_val, y_val, hyperparameters, seed):
    model = None
    model = build_LR(hyperparameters)
    try:
        earlystopping = tf.keras.callbacks.EarlyStopping(
            monitor=hyperparameters["monitor"],
            min_delta=hyperparameters["mindelta"],
            patience=hyperparameters["patience"],  # 30
            restore_best_weights=True,
            mode="min"
        )
    
        hist = model.fit(
            x=X_train, y=y_train,
            validation_data=(X_val, y_val),
            callbacks=[earlystopping], batch_size=hyperparameters['batch_size'], epochs=hyperparameters['epochs'],
            verbose=hyperparameters['verbose']
        )
                                
        return model, hist, earlystopping
    except KeyboardInterrupt:
        print('Training duration (s):', time.time() - global_start_time)
        return model, None, None

In [ ]:
from sklearn.model_selection import KFold
def myCVGrid_LR(X_prev_train, y_prev_train, hyperparameters, lr_scheduler, seed):
    bestHyperparameters = {}
    bestMetricDev = np.inf

    for j in range(len(lr_scheduler)):
        v_early = []
        v_metric_dev = []
        v_hist = []
        v_val_loss = []
        kf = KFold(n_splits=2)

        hyperparameters_copy = hyperparameters.copy()
        hyperparameters_copy['lr_scheduler'] = lr_scheduler[j]
        
        for train, val in kf.split(X_prev_train):
            #Load train and validation
            X_train = X_prev_train.iloc[train]
            y_train = y_prev_train.iloc[train]
            
            X_val = X_prev_train.iloc[val]
            y_val = y_prev_train.iloc[val]
            
            # Train the model 
            model, hist, earlystopping = run_network_LR(X_train, 
                                                        y_train, 
                                                        X_val, 
                                                        y_val, 
                                                        hyperparameters_copy, 
                                                        seed)
            
            
            v_early.append(earlystopping)
            v_hist.append(hist)
            v_val_loss.append(np.min(hist.history["val_loss"]))
            
            metric_dev = np.mean(v_val_loss)
            if metric_dev < bestMetricDev:
                bestMetricDev = metric_dev
                bestHyperparameters = {
                    'lr_scheduler': lr_scheduler[j]
                }

    return bestHyperparameters, X_train, y_train, X_val, y_val

### Auxiliary functions

In [ ]:
def save_results(split_directory, best_hyperparameters, y_test_pred, y_train_pred=None, model=None):

    if not os.path.exists(split_directory):
        os.makedirs(split_directory)

    with open(os.path.join(split_directory, "bestHyperparameters.pkl"), 'wb') as f:
        pickle.dump(best_hyperparameters, f)

    with open(os.path.join(split_directory, "y_test_pred.pkl"), 'wb') as f:
        pickle.dump(y_test_pred, f)
        
    if y_train_pred is not None:
        with open(os.path.join(split_directory, "y_train_pred.pkl"), 'wb') as f:
            pickle.dump(y_train_pred, f)

    if model is not None:
        model_filename = os.path.join(split_directory, "model.h5")
        model.save(model_filename)


# Hyperparameters

In [ ]:
seeds = [20, 30, 45, 70]

tensor = True
debug = True
balance = True

n_max_num = 5
n_categorical_features = 3
n_numerical_features = 5
n_static_features = n_categorical_features + n_numerical_features
n_dynamic_features = 56
n_timesteps = 14

# Hyperparamas of network
epochs = 10000
batch_size = 128

layers = [3, 5, 8, 10, 15, 20, 25, 30, 35, 40, 50]
lr_scheduler = [0.0001, 0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2, 0.3]

w2 = 0.18
w1 = 0.82

hyperparameters = {
    "n_categorical_features": n_categorical_features,
    "n_numerical_features": n_numerical_features,
    "n_static_features": n_static_features,
    "n_dynamic_features": n_dynamic_features,
    "n_timesteps": n_timesteps,
    "w1":w1, "w2":w2, 
    "epochs":epochs,
    'batch_size': batch_size,
    'maskValue':666,
    'monitor': 'val_loss', 
    "mindelta": 0,
    "patience":30,
    'balance': balance,
    'optimizer':'adam',
    'kfold':5,
    'level':3, 
    'verbose':0
}

# Code

In [ ]:
run_model = True

if run_model:
    v_early = []
    loss_dev_stat = []
    loss_dev_dyn = []
    loss_dev_LR = []
    v_models_stat = []
    v_models_dyn = []
    v_models_LR = []
    bestHyperparameters_bySplit_dyn = {}
    bestHyperparameters_bySplit_stat = {}
    bestHyperparameters_bySplit_LR = {}
    y_pred_by_split_stat = {}
    y_pred_by_split_dyn = {}
    y_pred_by_split_LR = {}


    for i in range(4):
        X_test_dynamic = np.load("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) + "/X_test_tensor.npy")
        X_test_static = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) + "/X_test_static.csv",
                                   index_col=0)
        y_test = pd.read_csv("../../ORIGINAL_DATA/splits_14_days/notbalanced/split_" + str(i) + "/y_test.csv",
                            index_col=0)

        # DYNAMIC #########################################################################################################
        bestHyperparameters_dyn, X_train, y_train, X_train_static, X_val, y_val, X_val_static = myCVGrid(hyperparameters,
                                                                                                     dropout_rate,
                                                                                                     lr_scheduler,
                                                                                                     layers,
                                                                                                     i,                                                              
                                                                                                     seeds[i],
                                                                                                     model_type="dynamic"
                                                                                                     )
        bestHyperparameters_bySplit_stat[str(i)] = bestHyperparameters_dyn

        hyperparameters.update({
            "dropout_rate": bestHyperparameters_dyn["dropout_rate"],
            "layers": bestHyperparameters_dyn["layers"],
            "lr_scheduler": bestHyperparameters_dyn["lr_scheduler"],
        })

        utils.reset_keras()
        model_dyn, hist, early = run_network(
            X_train, X_train_static, y_train.individualMRGerm.values,
            X_val, X_val_static, y_val.individualMRGerm.values,
            hyperparameters, seeds[i], model_type="dynamic"
        )

        #Save the hyperparameters and predictions
        split_directory = f'./Results_LFLR/Dynamic/split_{i}'
        y_pred_dynamic = model_dyn.predict(X_test_dynamic)
        y_train_pred_dynamic = model_dyn.predict(X_train)
        
        save_results(split_directory, bestHyperparameters_dyn, y_pred_dynamic, y_train_pred_dynamic, model_dyn)

        v_models_dyn.append(model_dyn)
        loss_dev_dyn.append(hist.history['val_loss'])
        y_pred_by_split_dyn[str(i)] = y_pred_dynamic

        # STATIC #########################################################################################################
        bestHyperparameters_stat, X_train, y_train, X_train_static, X_val, y_val, X_val_static = myCVGrid(hyperparameters,
                                                                                                     dropout_rate,
                                                                                                     lr_scheduler,
                                                                                                     layers,
                                                                                                     i,                                                              
                                                                                                     seeds[i],
                                                                                                     model_type="static"
                                                                                                     )
        bestHyperparameters_bySplit_dyn[str(i)] = bestHyperparameters_stat

        hyperparameters.update({
            "dropout_rate": bestHyperparameters_stat["dropout_rate"],
            "layers": bestHyperparameters_stat["layers"],
            "lr_scheduler": bestHyperparameters_stat["lr_scheduler"],
        })

        utils.reset_keras()
        model_stat, hist, early = run_network(
            X_train, X_train_static, y_train.individualMRGerm.values,
            X_val, X_val_static, y_val.individualMRGerm.values,
            hyperparameters, seeds[i], model_type="static"
        )

        #Save the hyperparameters and predictions
        split_directory = f'./Results_LFLR/Static/split_{i}'
        y_pred_static = model_stat.predict(X_test_static)
        y_train_pred_static = model_stat.predict(X_train_static.values)
        
        save_results(split_directory, bestHyperparameters_stat, y_pred_static, y_train_pred_static, model_stat)

        v_models_stat.append(model_stat)
        loss_dev_stat.append(hist.history['val_loss'])
        y_pred_by_split_stat[str(i)] = y_pred_static

        # LR #########################################################################################################
        y_train_summary = y_train.reset_index()
        y_train_summary["y_pred_static"] = y_train_pred_static
        y_train_summary["y_pred_dynamic"] = y_train_pred_dynamic    
        y_test_summary = y_test.reset_index()
        y_test_summary["y_pred_static"] = y_pred_static
        y_test_summary["y_pred_dynamic"] = y_pred_dynamic

        bestHyperparameters_LR, X_train_summary, y_train_summary_2, X_val_summary, y_val_summary_2, = myCVGrid_LR(y_train_summary[["y_pred_static", "y_pred_dynamic"]], 
                                                                                                                  y_train_summary[["individualMRGerm"]],
                                                                                                                  hyperparameters,
                                                                                                                  lr_scheduler,
                                                                                                                  seeds[i],
                                                                                                                 )

        bestHyperparameters_bySplit_LR[str(i)] = bestHyperparameters_LR

        hyperparameters.update({
            "lr_scheduler": bestHyperparameters_LR["lr_scheduler"],
        })

        utils.reset_keras()
        model_LR, hist, earlystopping = run_network_LR(X_train_summary, 
                                                    y_train_summary_2, 
                                                    X_val_summary, 
                                                    y_val_summary_2, 
                                                    hyperparameters, 
                                                    seeds[i])
        
        y_pred = model_LR.predict(y_test_summary[["y_pred_static", "y_pred_dynamic"]])

        split_directory = f'./Results_LFLR/LR/split_{i}'
        save_results(split_directory, bestHyperparameters_LR, y_pred, None, model_LR)

        v_models_stat.append(model_LR)
        loss_dev_LR.append(hist.history['val_loss'])
        y_pred_by_split_LR[str(i)] = y_pred

        # Calculate metrics
        metrics_dict = utils.calculate_and_save_metrics(
        y_test.individualMRGerm.values, 
        y_pred, 
        split_directory, 
        split_index=i
        )